For each color pair, partitions the card pool into disjoint subsets ("communities") using https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html

Within each community, sorts cards from highest to lowest centrality (importance within that community).

Outputs results into `colorpairs.html` file in this directory.

In [ ]:
from mtg_draft_ai.api import *

cards = read_cube_toml('cube_81183_tag_data.toml')

In [ ]:
from mtg_draft_ai import synergy

G = synergy.create_graph(cards)

In [ ]:
def contains_all_colors(cards, colors):
    colors_seen = set()
    for card in cards:
        if card.color_id != 'C':
            for color in card.color_id:
                colors_seen.add(color)
    
    return colors_seen == set(colors)

In [ ]:
import networkx as nx
from networkx.algorithms import community
from mtg_draft_ai.display import cards_to_html

output_html = """
<style>
card-image {
    display: inline;
    margin: 1px;
}
</style>
"""

color_pairs = ['WU', 'WB', 'WR', 'WG', 'UB', 'UR', 'UG', 'BR', 'BG', 'RG']
color_trios = ['WUB', 'UBR', 'BRG', 'RGW', 'GWU', 'WBR', 'URG', 'BGW', 'RWU', 'GUB']

for colors in color_pairs:
    output_html += 'Colors: {} <br>\n'.format(colors)
    subgraph = synergy.colors_subgraph(G, colors)
    colors_gmc = community.greedy_modularity_communities(subgraph)

    i = 0
    for gmc in colors_gmc:
        if not contains_all_colors(gmc, colors):
            continue

        gmc_subgraph = subgraph.subgraph(gmc)
        sorted_community_cards = [k for k, v in synergy.sorted_centralities(gmc_subgraph)]
        
        output_html += 'Community {}\n'.format(i)
        output_html += '<div>\n{}</div>\n'.format(cards_to_html(sorted_community_cards))
        i += 1

#print(output_html)

In [ ]:
with open('colorpairs.html', 'w') as f:
    f.write(output_html)